# 📊 Q-VAL PETR4 — Data Ingestion

**Notebook**: `01_data_ingestion.ipynb`  
**Domínio**: Infraestrutura / Ingestão de Dados  
**Responsabilidade**: Coleta, persistência e validação de dados de mercado de múltiplas fontes.

## Fontes de Dados

| Dado | Fonte | API |
|:-----|:------|:----|
| PETR4 (preços, fundamentalistas) | Brapi | `brapi.dev/api` |
| IBOV (benchmark) | Yahoo Finance | `yfinance` |
| SELIC (taxa livre de risco) | BCB | `api.bcb.gov.br/SGS` |

## Mapeamento de Artefatos

| Codeblock | Artefato | Destino |
|:---------:|:---------|:--------|
| 1 | Configuração Global | (variáveis em memória) |
| 2 | `petr4_quote.json` | `data/external/brapi/` |
| 3 | `petr4_historical.json`, `petr4_prices.csv` | `data/external/brapi/`, `data/processed/` |
| 4 | `petr4_fundamental.json`, `petr4_dividends.csv` | `data/external/brapi/`, `data/processed/` |
| 5 | `petr4_financial_data.json` | `data/external/brapi/` |
| 6 | `petr4_balance_sheet.json`, `.csv` | `data/external/brapi/`, `data/processed/` |
| 7 | `petr4_income_statement.json`, `.csv` | `data/external/brapi/`, `data/processed/` |
| 8 | `petr4_cashflow.json`, `.csv` | `data/external/brapi/`, `data/processed/` |
| 9 | `ibov_historical.json`, `ibov_prices.csv` | `data/external/yfinance/`, `data/processed/` |
| 9.1 | `selic_meta.json`, `selic.csv` | `data/external/bcb/`, `data/processed/` |
| 10 | `returns.csv` | `data/processed/` |
| 11 | `collection_log.json` | `data/outputs/` |

---

## Fluxo de Dados

```
Brapi API    → data/external/brapi/*.json (raw)
Yahoo Finance → data/external/yfinance/*.json (raw)
BCB/SGS      → data/external/bcb/*.json (raw)
             ↓
         data/processed/*.csv (processado)
             ↓
         data/outputs/ (artefatos finais)
```

## 1. Configuração Global (Codeblock de Config)

**Objetivo**: Definir todas as variáveis de configuração em um único local centralizado.  
**Artefato**: Variáveis em memória (não persiste arquivo).

### Variáveis Configuráveis:
- `BRAPI_BASE_URL`: URL base da API Brapi
- `BRAPI_TOKEN`: Token de autenticação (vazio para tickers de teste)
- `TICKER_PRINCIPAL`: Ativo principal de análise (PETR4)
- `TICKER_BENCHMARK`: Índice de benchmark (^BVSP)
- `DATA_RANGE`: Período de dados históricos
- `DATA_INTERVAL`: Granularidade dos dados
- `DATA_REFERENCIA`: Data de referência para a análise
- `PATH_*`: Caminhos para persistência de dados

In [1]:
# ==============================================================================
# CODEBLOCK 1: CONFIGURAÇÃO GLOBAL
# ==============================================================================
# Este é o ÚNICO codeblock de configuração. Todas as variáveis alteráveis
# estão centralizadas aqui. Modifique conforme necessário antes de executar.
# ==============================================================================

import os
import json
import requests
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
from pathlib import Path

# ------------------------------------------------------------------------------
# CONFIGURAÇÃO DA API BRAPI
# ------------------------------------------------------------------------------
BRAPI_BASE_URL = "https://brapi.dev/api"
BRAPI_TOKEN = ""  # Vazio para tickers de teste (PETR4, VALE3, ITUB4, MGLU3)

# ------------------------------------------------------------------------------
# CONFIGURAÇÃO DE TICKERS
# ------------------------------------------------------------------------------
TICKER_PRINCIPAL = "PETR4"  # Ativo principal de análise
TICKER_BENCHMARK = "^BVSP"  # Índice de benchmark (IBOVESPA)

# ------------------------------------------------------------------------------
# CONFIGURAÇÃO DE PERÍODO
# ------------------------------------------------------------------------------
DATA_INICIO = "2016-01-01"  # Data inicial fixa
DATA_INTERVAL = "1d"  # Intervalo: 1m, 5m, 15m, 30m, 1h, 1d, 5d, 1wk, 1mo
DATA_REFERENCIA = datetime.now().strftime("%Y-%m-%d")

# Range para API Brapi (calculado a partir da data inicial)
dias_range = (datetime.now() - datetime.strptime(DATA_INICIO, "%Y-%m-%d")).days
RANGE_MAPPING = {
    "1d": 1, "5d": 5, "1mo": 30, "3mo": 90, "6mo": 180,
    "1y": 365, "2y": 730, "5y": 1825, "10y": 3650, "max": 7300
}
DATA_RANGE = "10y" if dias_range > 1825 else "5y"

# ------------------------------------------------------------------------------
# CONFIGURAÇÃO DE CAMINHOS
# ------------------------------------------------------------------------------
# Base path (relativo ao notebook)
BASE_PATH = Path("..").resolve()

# Paths de saída
PATH_EXTERNAL = BASE_PATH / "data" / "external" / "brapi"
PATH_EXTERNAL_BCB = BASE_PATH / "data" / "external" / "bcb"
PATH_EXTERNAL_YFINANCE = BASE_PATH / "data" / "external" / "yfinance"
PATH_RAW = BASE_PATH / "data" / "raw"
PATH_PROCESSED = BASE_PATH / "data" / "processed"
PATH_OUTPUTS = BASE_PATH / "data" / "outputs"
PATH_OUTPUTS_TABLES = PATH_OUTPUTS / "tables"
PATH_OUTPUTS_FIGURES = PATH_OUTPUTS / "figures"

# ------------------------------------------------------------------------------
# CRIAÇÃO DA ESTRUTURA DE DIRETÓRIOS
# ------------------------------------------------------------------------------
for path in [PATH_EXTERNAL, PATH_EXTERNAL_BCB, PATH_EXTERNAL_YFINANCE, PATH_RAW, 
             PATH_PROCESSED, PATH_OUTPUTS_TABLES, PATH_OUTPUTS_FIGURES]:
    path.mkdir(parents=True, exist_ok=True)

# ------------------------------------------------------------------------------
# FUNÇÕES AUXILIARES
# ------------------------------------------------------------------------------
def brapi_get(endpoint: str, params: dict = None) -> dict:
    """
    Faz requisição GET à API Brapi.
    
    Args:
        endpoint: Caminho do endpoint (ex: "/quote/PETR4")
        params: Parâmetros da query string
    
    Returns:
        dict: Resposta JSON da API
    """
    url = f"{BRAPI_BASE_URL}{endpoint}"
    if params is None:
        params = {}
    if BRAPI_TOKEN:
        params["token"] = BRAPI_TOKEN
    
    response = requests.get(url, params=params, timeout=30)
    response.raise_for_status()
    return response.json()

def save_json(data: dict, filepath: Path, add_metadata: bool = True) -> None:
    """
    Salva dados em arquivo JSON com metadata de coleta.
    
    Args:
        data: Dados a serem salvos
        filepath: Caminho do arquivo
        add_metadata: Se True, adiciona timestamp de coleta
    """
    if add_metadata:
        data = {
            "_metadata": {
                "collected_at": datetime.now().isoformat(),
                "data_referencia": DATA_REFERENCIA,
                "source": "brapi.dev"
            },
            **data
        }
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✅ Salvo: {filepath}")

def load_json(filepath: Path) -> dict:
    """Carrega dados de arquivo JSON."""
    with open(filepath, "r", encoding="utf-8") as f:
        return json.load(f)

# ------------------------------------------------------------------------------
# VALIDAÇÃO DA CONFIGURAÇÃO
# ------------------------------------------------------------------------------
print("=" * 60)
print("🔧 CONFIGURAÇÃO Q-VAL - COLETA BRAPI + YFINANCE")
print("=" * 60)
print(f"📅 Data de Referência: {DATA_REFERENCIA}")
print(f"📅 Data Início: {DATA_INICIO}")
print(f"🎯 Ticker Principal: {TICKER_PRINCIPAL} (Brapi)")
print(f"📈 Benchmark: {TICKER_BENCHMARK} (Yahoo Finance)")
print(f"📆 Período: {DATA_RANGE} | Intervalo: {DATA_INTERVAL}")
print(f"🔑 Token Brapi: {'Configurado' if BRAPI_TOKEN else 'Não configurado (usando tickers de teste)'}")
print("=" * 60)
print(f"📁 Paths configurados:")
print(f"   External Brapi: {PATH_EXTERNAL}")
print(f"   External YFinance: {PATH_EXTERNAL_YFINANCE}")
print(f"   Processed: {PATH_PROCESSED}")
print(f"   Outputs: {PATH_OUTPUTS}")
print("=" * 60)

🔧 CONFIGURAÇÃO Q-VAL - COLETA BRAPI + YFINANCE
📅 Data de Referência: 2025-12-06
📅 Data Início: 2016-01-01
🎯 Ticker Principal: PETR4 (Brapi)
📈 Benchmark: ^BVSP (Yahoo Finance)
📆 Período: 10y | Intervalo: 1d
🔑 Token Brapi: Não configurado (usando tickers de teste)
📁 Paths configurados:
   External Brapi: /home/subtilizer/Workspace/unb-cca-mqac/data/external/brapi
   External YFinance: /home/subtilizer/Workspace/unb-cca-mqac/data/external/yfinance
   Processed: /home/subtilizer/Workspace/unb-cca-mqac/data/processed
   Outputs: /home/subtilizer/Workspace/unb-cca-mqac/data/outputs


## 2. Coleta de Cotação Atual — PETR4

**Endpoint**: `GET /api/quote/PETR4`  
**Artefato**: `data/external/brapi/petr4_quote.json`

### Campos validados:
- `regularMarketPrice`: Preço atual
- `regularMarketChange`: Variação absoluta
- `regularMarketChangePercent`: Variação percentual
- `marketCap`: Capitalização de mercado
- `fiftyTwoWeekHigh`, `fiftyTwoWeekLow`: Máxima/mínima 52 semanas

In [2]:
# ==============================================================================
# CODEBLOCK 2: COLETA DE COTAÇÃO ATUAL — PETR4
# ==============================================================================
# Artefato: data/external/brapi/petr4_quote.json
# ==============================================================================

print(f"📡 Coletando cotação atual de {TICKER_PRINCIPAL}...")

# Chamar endpoint
data = brapi_get(f"/quote/{TICKER_PRINCIPAL}")

# Validar resposta
if "results" not in data or len(data["results"]) == 0:
    raise ValueError(f"Nenhum resultado retornado para {TICKER_PRINCIPAL}")

result = data["results"][0]

# Campos obrigatórios para validação
required_fields = [
    "regularMarketPrice",
    "regularMarketChange", 
    "regularMarketChangePercent",
    "marketCap",
    "fiftyTwoWeekHigh",
    "fiftyTwoWeekLow",
    "symbol"
]

# Validar campos
missing_fields = [f for f in required_fields if f not in result or result[f] is None]
if missing_fields:
    print(f"⚠️ Campos ausentes: {missing_fields}")

# Salvar JSON
save_json(data, PATH_EXTERNAL / "petr4_quote.json")

# Exibir resumo
print("\n📊 Resumo da Cotação:")
print(f"   Ticker: {result.get('symbol')}")
print(f"   Preço Atual: R$ {result.get('regularMarketPrice', 'N/A'):.2f}")
print(f"   Variação: {result.get('regularMarketChangePercent', 'N/A'):.2f}%")
print(f"   Market Cap: R$ {result.get('marketCap', 0)/1e9:.2f} bi")
print(f"   52wk High: R$ {result.get('fiftyTwoWeekHigh', 'N/A')}")
print(f"   52wk Low: R$ {result.get('fiftyTwoWeekLow', 'N/A')}")

📡 Coletando cotação atual de PETR4...
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/external/brapi/petr4_quote.json

📊 Resumo da Cotação:
   Ticker: PETR4
   Preço Atual: R$ 31.37
   Variação: -3.54%
   Market Cap: R$ 421.66 bi
   52wk High: R$ 40.76
   52wk Low: R$ 28.86


## 3. Coleta de Histórico de Preços — PETR4 (5 anos)

**Endpoint**: `GET /api/quote/PETR4?range=5y&interval=1d`  
**Artefatos**:
- Raw: `data/external/brapi/petr4_historical.json`
- Processado: `data/processed/petr4_prices.csv`

### Campos extraídos de `historicalDataPrice`:
- `date`: Timestamp Unix → convertido para datetime
- `open`, `high`, `low`, `close`: Preços OHLC
- `adjustedClose`: Preço ajustado para proventos
- `volume`: Volume negociado

In [3]:
# ==============================================================================
# CODEBLOCK 3: COLETA DE HISTÓRICO DE PREÇOS — PETR4
# ==============================================================================
# Artefatos:
#   - data/external/brapi/petr4_historical.json (raw)
#   - data/processed/petr4_prices.csv (processado)
# ==============================================================================

print(f"📡 Coletando histórico de preços de {TICKER_PRINCIPAL} ({DATA_RANGE})...")

# Chamar endpoint com parâmetros de período
data = brapi_get(f"/quote/{TICKER_PRINCIPAL}", params={
    "range": DATA_RANGE,
    "interval": DATA_INTERVAL
})

# Validar resposta
if "results" not in data or len(data["results"]) == 0:
    raise ValueError(f"Nenhum resultado retornado para {TICKER_PRINCIPAL}")

result = data["results"][0]

# Validar dados históricos
if "historicalDataPrice" not in result or len(result["historicalDataPrice"]) == 0:
    raise ValueError("Nenhum dado histórico retornado")

# Salvar JSON raw
save_json(data, PATH_EXTERNAL / "petr4_historical.json")

# Processar para CSV
historical_data = result["historicalDataPrice"]
df = pd.DataFrame(historical_data)

# Converter timestamp Unix para datetime
df["date"] = pd.to_datetime(df["date"], unit="s")
df["date"] = df["date"].dt.strftime("%Y-%m-%d")

# Renomear colunas para padrão
df = df.rename(columns={
    "adjustedClose": "adj_close"
})

# Selecionar e ordenar colunas
columns = ["date", "open", "high", "low", "close", "adj_close", "volume"]
df = df[columns].sort_values("date").reset_index(drop=True)

# Salvar CSV processado
csv_path = PATH_PROCESSED / "petr4_prices.csv"
df.to_csv(csv_path, index=False)
print(f"✅ Salvo: {csv_path}")

# Exibir resumo
print(f"\n📊 Resumo do Histórico:")
print(f"   Período: {df['date'].min()} a {df['date'].max()}")
print(f"   Total de registros: {len(df)}")
print(f"   Preço inicial: R$ {df['close'].iloc[0]:.2f}")
print(f"   Preço final: R$ {df['close'].iloc[-1]:.2f}")
print(f"   Retorno total: {((df['close'].iloc[-1] / df['close'].iloc[0]) - 1) * 100:.2f}%")
print(f"\n   Primeiros registros:")
print(df.head().to_string(index=False))

📡 Coletando histórico de preços de PETR4 (10y)...
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/external/brapi/petr4_historical.json
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/processed/petr4_prices.csv

📊 Resumo do Histórico:
   Período: 2015-12-09 a 2025-12-05
   Total de registros: 2490
   Preço inicial: R$ 7.65
   Preço final: R$ 31.37
   Retorno total: 310.07%

   Primeiros registros:
      date  open  high  low  close  adj_close   volume
2015-12-09  7.34  7.82 7.27   7.65     2.1025 82102800
2015-12-10  7.51  7.57 7.31   7.45     2.0476 43458400
2015-12-11  7.26  7.53 7.21   7.25     1.9926 50750600
2015-12-14  7.15  7.23 7.07   7.21     1.9816 33538000
2015-12-15  7.34  7.47 7.30   7.42     2.0393 34188900


## 4. Coleta de Dados Fundamentalistas com Dividendos — PETR4

**Endpoint**: `GET /api/quote/PETR4?fundamental=true&dividends=true`  
**Artefatos**:
- Raw: `data/external/brapi/petr4_fundamental.json`
- Processado: `data/processed/petr4_dividends.csv`

### Campos extraídos:
- `priceEarnings`: P/L (Preço/Lucro)
- `earningsPerShare`: LPA (Lucro por Ação)
- `dividendsData.cashDividends[]`: Histórico de dividendos/JCP

In [4]:
# ==============================================================================
# CODEBLOCK 4: COLETA DE DADOS FUNDAMENTALISTAS COM DIVIDENDOS — PETR4
# ==============================================================================
# Artefatos:
#   - data/external/brapi/petr4_fundamental.json (raw)
#   - data/processed/petr4_dividends.csv (processado)
# ==============================================================================

print(f"📡 Coletando dados fundamentalistas e dividendos de {TICKER_PRINCIPAL}...")

# Chamar endpoint
data = brapi_get(f"/quote/{TICKER_PRINCIPAL}", params={
    "fundamental": "true",
    "dividends": "true"
})

# Validar resposta
if "results" not in data or len(data["results"]) == 0:
    raise ValueError(f"Nenhum resultado retornado para {TICKER_PRINCIPAL}")

result = data["results"][0]

# Salvar JSON raw
save_json(data, PATH_EXTERNAL / "petr4_fundamental.json")

# Extrair métricas fundamentalistas
pe_ratio = result.get("priceEarnings")
eps = result.get("earningsPerShare")

print(f"\n📊 Métricas Fundamentalistas:")
print(f"   P/L (priceEarnings): {pe_ratio:.2f}" if pe_ratio else "   P/L: N/A")
print(f"   LPA (earningsPerShare): R$ {eps:.4f}" if eps else "   LPA: N/A")

# Processar dividendos
dividends_data = result.get("dividendsData", {})
cash_dividends = dividends_data.get("cashDividends", [])

if cash_dividends:
    df_dividends = pd.DataFrame(cash_dividends)
    
    # Selecionar e renomear colunas
    df_dividends = df_dividends[["paymentDate", "rate", "label", "relatedTo"]].copy()
    df_dividends.columns = ["payment_date", "rate", "label", "related_to"]
    
    # Converter datas (com tratamento de erros)
    df_dividends["payment_date"] = pd.to_datetime(df_dividends["payment_date"], errors="coerce")
    
    # Remover linhas com datas inválidas
    df_dividends = df_dividends.dropna(subset=["payment_date"])
    df_dividends["payment_date"] = df_dividends["payment_date"].dt.strftime("%Y-%m-%d")
    
    # Ordenar por data
    df_dividends = df_dividends.sort_values("payment_date", ascending=False).reset_index(drop=True)
    
    # Salvar CSV
    csv_path = PATH_PROCESSED / "petr4_dividends.csv"
    df_dividends.to_csv(csv_path, index=False)
    print(f"✅ Salvo: {csv_path}")
    
    # Exibir resumo
    print(f"\n💰 Histórico de Dividendos/JCP:")
    print(f"   Total de eventos: {len(df_dividends)}")
    if len(df_dividends) > 0:
        print(f"   Período: {df_dividends['payment_date'].min()} a {df_dividends['payment_date'].max()}")
        print(f"\n   Últimos 5 eventos:")
        print(df_dividends.head().to_string(index=False))
else:
    print("\n⚠️ Nenhum dado de dividendos disponível")

📡 Coletando dados fundamentalistas e dividendos de PETR4...
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/external/brapi/petr4_fundamental.json

📊 Métricas Fundamentalistas:
   P/L (priceEarnings): 5.37
   LPA (earningsPerShare): R$ 6.0147
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/processed/petr4_dividends.csv

💰 Histórico de Dividendos/JCP:
   Total de eventos: 50
   Período: 2018-08-22 a 2025-09-22

   Últimos 5 eventos:
payment_date    rate     label    related_to
  2025-09-22 0.30845 DIVIDENDO Setembro/2025
  2025-09-22 0.14614       JCP Setembro/2025
  2025-08-20 0.45458       JCP   Agosto/2025
  2025-06-20 0.35477 DIVIDENDO    Junho/2025
  2025-03-20 0.65357 DIVIDENDO    Março/2025


## 5. Coleta de Módulos Financeiros — PETR4

**Endpoint**: `GET /api/quote/PETR4?modules=financialData,defaultKeyStatistics,summaryProfile`  
**Artefato**: `data/external/brapi/petr4_financial_data.json`

### Módulos e campos extraídos:

**financialData** (TTM):
- `ebitda`, `ebitdaMargins`, `operatingMargins`, `profitMargins`
- `returnOnAssets` (ROA), `returnOnEquity` (ROE)
- `currentRatio`, `quickRatio`, `debtToEquity`
- `totalRevenue`, `grossProfits`, `freeCashflow`

**defaultKeyStatistics** (TTM):
- `enterpriseValue`, `enterpriseToEbitda` (EV/EBITDA)
- `priceToBook` (P/VP), `bookValue` (VPA)
- `beta`, `dividendYield`
- `sharesOutstanding`, `floatShares`

**summaryProfile**:
- `sector`, `industry`, `longBusinessSummary`

In [5]:
# ==============================================================================
# CODEBLOCK 5: COLETA DE MÓDULOS FINANCEIROS — PETR4
# ==============================================================================
# Artefato: data/external/brapi/petr4_financial_data.json
# ==============================================================================

print(f"📡 Coletando módulos financeiros de {TICKER_PRINCIPAL}...")

# Módulos a serem coletados
modules = "financialData,defaultKeyStatistics,summaryProfile"

# Chamar endpoint
data = brapi_get(f"/quote/{TICKER_PRINCIPAL}", params={"modules": modules})

# Validar resposta
if "results" not in data or len(data["results"]) == 0:
    raise ValueError(f"Nenhum resultado retornado para {TICKER_PRINCIPAL}")

result = data["results"][0]

# Salvar JSON raw
save_json(data, PATH_EXTERNAL / "petr4_financial_data.json")

# Extrair e exibir financialData
financial_data = result.get("financialData", {})
print("\n📊 Financial Data (TTM):")
financial_metrics = {
    "EBITDA": ("ebitda", lambda x: f"R$ {x/1e9:.2f} bi" if x else "N/A"),
    "Margem EBITDA": ("ebitdaMargins", lambda x: f"{x*100:.2f}%" if x else "N/A"),
    "Margem Operacional": ("operatingMargins", lambda x: f"{x*100:.2f}%" if x else "N/A"),
    "Margem Líquida": ("profitMargins", lambda x: f"{x*100:.2f}%" if x else "N/A"),
    "ROA": ("returnOnAssets", lambda x: f"{x*100:.2f}%" if x else "N/A"),
    "ROE": ("returnOnEquity", lambda x: f"{x*100:.2f}%" if x else "N/A"),
    "Liquidez Corrente": ("currentRatio", lambda x: f"{x:.2f}" if x else "N/A"),
    "Liquidez Seca": ("quickRatio", lambda x: f"{x:.2f}" if x else "N/A"),
    "Dívida/PL": ("debtToEquity", lambda x: f"{x:.2f}%" if x else "N/A"),
    "Receita Total": ("totalRevenue", lambda x: f"R$ {x/1e9:.2f} bi" if x else "N/A"),
    "FCL": ("freeCashflow", lambda x: f"R$ {x/1e9:.2f} bi" if x else "N/A"),
}
for label, (key, formatter) in financial_metrics.items():
    value = financial_data.get(key)
    print(f"   {label}: {formatter(value)}")

# Extrair e exibir defaultKeyStatistics
key_stats = result.get("defaultKeyStatistics", {})
print("\n📊 Key Statistics:")
stats_metrics = {
    "Enterprise Value": ("enterpriseValue", lambda x: f"R$ {x/1e9:.2f} bi" if x else "N/A"),
    "EV/EBITDA": ("enterpriseToEbitda", lambda x: f"{x:.2f}x" if x else "N/A"),
    "P/VP": ("priceToBook", lambda x: f"{x:.2f}x" if x else "N/A"),
    "VPA": ("bookValue", lambda x: f"R$ {x:.2f}" if x else "N/A"),
    "Beta": ("beta", lambda x: f"{x:.2f}" if x else "N/A"),
    "Dividend Yield": ("dividendYield", lambda x: f"{x*100:.2f}%" if x else "N/A"),
    "Ações em Circulação": ("sharesOutstanding", lambda x: f"{x/1e9:.2f} bi" if x else "N/A"),
}
for label, (key, formatter) in stats_metrics.items():
    value = key_stats.get(key)
    print(f"   {label}: {formatter(value)}")

# Extrair e exibir summaryProfile
profile = result.get("summaryProfile", {})
print("\n📊 Summary Profile:")
print(f"   Setor: {profile.get('sector', 'N/A')}")
print(f"   Indústria: {profile.get('industry', 'N/A')}")
print(f"   Funcionários: {profile.get('fullTimeEmployees', 'N/A'):,}" if profile.get('fullTimeEmployees') else "   Funcionários: N/A")

📡 Coletando módulos financeiros de PETR4...
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/external/brapi/petr4_financial_data.json

📊 Financial Data (TTM):
   EBITDA: R$ 214.17 bi
   Margem EBITDA: 43.79%
   Margem Operacional: 27.47%
   Margem Líquida: 7.60%
   ROA: 3.07%
   ROE: 8.75%
   Liquidez Corrente: 0.82
   Liquidez Seca: 0.34
   Dívida/PL: 88.50%
   Receita Total: R$ 489.15 bi
   FCL: R$ 123.18 bi

📊 Key Statistics:
   Enterprise Value: R$ 421.46 bi
   EV/EBITDA: 1.97x
   P/VP: 0.99x
   VPA: R$ 32.97
   Beta: N/A
   Dividend Yield: 1589.32%
   Ações em Circulação: 12.89 bi

📊 Summary Profile:
   Setor: Energy
   Indústria: Oil & Gas Integrated
   Funcionários: 45,149


## 6. Coleta de Balanço Patrimonial — PETR4

**Endpoint**: `GET /api/quote/PETR4?modules=balanceSheetHistory,balanceSheetHistoryQuarterly`  
**Artefatos**:
- Raw: `data/external/brapi/petr4_balance_sheet.json`
- Processado: `data/processed/petr4_balance_sheet.csv`

### Campos extraídos:
- `totalAssets`: Ativo Total
- `totalCurrentAssets`: Ativo Circulante
- `totalLiab`: Passivo Total
- `totalStockholderEquity`: Patrimônio Líquido
- `longTermDebt`: Dívida de Longo Prazo
- `shortLongTermDebt`: Dívida de Curto Prazo
- `cash`: Caixa e Equivalentes
- `totalCurrentLiabilities`: Passivo Circulante
- `propertyPlantEquipment`: Imobilizado

In [6]:
# ==============================================================================
# CODEBLOCK 6: COLETA DE BALANÇO PATRIMONIAL — PETR4
# ==============================================================================
# Artefatos:
#   - data/external/brapi/petr4_balance_sheet.json (raw)
#   - data/processed/petr4_balance_sheet.csv (processado)
# ==============================================================================

print(f"📡 Coletando Balanço Patrimonial de {TICKER_PRINCIPAL}...")

# Módulos a serem coletados
modules = "balanceSheetHistory,balanceSheetHistoryQuarterly"

# Chamar endpoint
data = brapi_get(f"/quote/{TICKER_PRINCIPAL}", params={"modules": modules})

# Validar resposta
if "results" not in data or len(data["results"]) == 0:
    raise ValueError(f"Nenhum resultado retornado para {TICKER_PRINCIPAL}")

result = data["results"][0]

# Salvar JSON raw
save_json(data, PATH_EXTERNAL / "petr4_balance_sheet.json")

# Campos a extrair
bs_fields = [
    "endDate", "type", "totalAssets", "totalCurrentAssets", "totalLiab",
    "totalStockholderEquity", "longTermDebt", "shortLongTermDebt", "cash",
    "totalCurrentLiabilities", "propertyPlantEquipment"
]

# Processar histórico anual
annual_data = result.get("balanceSheetHistory", [])
quarterly_data = result.get("balanceSheetHistoryQuarterly", [])

all_records = []

# Processar dados anuais
for record in annual_data:
    row = {field: record.get(field) for field in bs_fields}
    row["period_type"] = "annual"
    all_records.append(row)

# Processar dados trimestrais
for record in quarterly_data:
    row = {field: record.get(field) for field in bs_fields}
    row["period_type"] = "quarterly"
    all_records.append(row)

if all_records:
    df_bs = pd.DataFrame(all_records)
    
    # Renomear colunas
    df_bs = df_bs.rename(columns={
        "endDate": "end_date",
        "type": "report_type",
        "totalAssets": "total_assets",
        "totalCurrentAssets": "current_assets",
        "totalLiab": "total_liabilities",
        "totalStockholderEquity": "equity",
        "longTermDebt": "long_term_debt",
        "shortLongTermDebt": "short_term_debt",
        "totalCurrentLiabilities": "current_liabilities",
        "propertyPlantEquipment": "fixed_assets"
    })
    
    # Ordenar por data
    df_bs = df_bs.sort_values("end_date", ascending=False).reset_index(drop=True)
    
    # Salvar CSV
    csv_path = PATH_PROCESSED / "petr4_balance_sheet.csv"
    df_bs.to_csv(csv_path, index=False)
    print(f"✅ Salvo: {csv_path}")
    
    # Exibir resumo
    print(f"\n📊 Balanço Patrimonial:")
    print(f"   Total de registros: {len(df_bs)}")
    print(f"   Anuais: {len(annual_data)} | Trimestrais: {len(quarterly_data)}")
    
    # Último balanço anual
    latest_annual = df_bs[df_bs["period_type"] == "annual"].iloc[0] if len(annual_data) > 0 else None
    if latest_annual is not None:
        print(f"\n   Último Balanço Anual ({latest_annual['end_date']}):")
        print(f"   Ativo Total: R$ {latest_annual['total_assets']/1e9:.2f} bi" if latest_annual['total_assets'] else "   Ativo Total: N/A")
        print(f"   Patrimônio Líquido: R$ {latest_annual['equity']/1e9:.2f} bi" if latest_annual['equity'] else "   Patrimônio Líquido: N/A")
        print(f"   Dívida LP: R$ {latest_annual['long_term_debt']/1e9:.2f} bi" if latest_annual['long_term_debt'] else "   Dívida LP: N/A")
else:
    print("⚠️ Nenhum dado de Balanço Patrimonial disponível")

📡 Coletando Balanço Patrimonial de PETR4...
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/external/brapi/petr4_balance_sheet.json
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/processed/petr4_balance_sheet.csv

📊 Balanço Patrimonial:
   Total de registros: 77
   Anuais: 16 | Trimestrais: 61

   Último Balanço Anual (2024-12-31):
   Ativo Total: R$ 1124.80 bi
   Patrimônio Líquido: R$ 367.51 bi
   Dívida LP: R$ 304.68 bi


## 7. Coleta de DRE — PETR4

**Endpoint**: `GET /api/quote/PETR4?modules=incomeStatementHistory,incomeStatementHistoryQuarterly`  
**Artefatos**:
- Raw: `data/external/brapi/petr4_income_statement.json`
- Processado: `data/processed/petr4_income_statement.csv`

### Campos extraídos:
- `totalRevenue`: Receita Líquida
- `grossProfit`: Lucro Bruto
- `operatingIncome`: Lucro Operacional
- `netIncome`: Lucro Líquido
- `ebit`: EBIT
- `interestExpense`: Despesas Financeiras
- `incomeTaxExpense`: IR/CSLL

In [7]:
# ==============================================================================
# CODEBLOCK 7: COLETA DE DRE — PETR4
# ==============================================================================
# Artefatos:
#   - data/external/brapi/petr4_income_statement.json (raw)
#   - data/processed/petr4_income_statement.csv (processado)
# ==============================================================================

print(f"📡 Coletando DRE de {TICKER_PRINCIPAL}...")

# Módulos a serem coletados
modules = "incomeStatementHistory,incomeStatementHistoryQuarterly"

# Chamar endpoint
data = brapi_get(f"/quote/{TICKER_PRINCIPAL}", params={"modules": modules})

# Validar resposta
if "results" not in data or len(data["results"]) == 0:
    raise ValueError(f"Nenhum resultado retornado para {TICKER_PRINCIPAL}")

result = data["results"][0]

# Salvar JSON raw
save_json(data, PATH_EXTERNAL / "petr4_income_statement.json")

# Campos a extrair
is_fields = [
    "endDate", "type", "totalRevenue", "grossProfit", "operatingIncome",
    "netIncome", "netIncomeApplicableToCommonShares", "ebit",
    "interestExpense", "incomeTaxExpense"
]

# Processar histórico anual e trimestral
annual_data = result.get("incomeStatementHistory", [])
quarterly_data = result.get("incomeStatementHistoryQuarterly", [])

all_records = []

for record in annual_data:
    row = {field: record.get(field) for field in is_fields}
    row["period_type"] = "annual"
    all_records.append(row)

for record in quarterly_data:
    row = {field: record.get(field) for field in is_fields}
    row["period_type"] = "quarterly"
    all_records.append(row)

if all_records:
    df_is = pd.DataFrame(all_records)
    
    # Renomear colunas
    df_is = df_is.rename(columns={
        "endDate": "end_date",
        "type": "report_type",
        "totalRevenue": "revenue",
        "grossProfit": "gross_profit",
        "operatingIncome": "operating_income",
        "netIncome": "net_income",
        "netIncomeApplicableToCommonShares": "net_income_common",
        "interestExpense": "interest_expense",
        "incomeTaxExpense": "income_tax"
    })
    
    # Ordenar por data
    df_is = df_is.sort_values("end_date", ascending=False).reset_index(drop=True)
    
    # Salvar CSV
    csv_path = PATH_PROCESSED / "petr4_income_statement.csv"
    df_is.to_csv(csv_path, index=False)
    print(f"✅ Salvo: {csv_path}")
    
    # Exibir resumo
    print(f"\n📊 DRE:")
    print(f"   Total de registros: {len(df_is)}")
    print(f"   Anuais: {len(annual_data)} | Trimestrais: {len(quarterly_data)}")
    
    # Última DRE anual
    latest_annual = df_is[df_is["period_type"] == "annual"].iloc[0] if len(annual_data) > 0 else None
    if latest_annual is not None:
        print(f"\n   Última DRE Anual ({latest_annual['end_date']}):")
        print(f"   Receita: R$ {latest_annual['revenue']/1e9:.2f} bi" if latest_annual['revenue'] else "   Receita: N/A")
        print(f"   Lucro Bruto: R$ {latest_annual['gross_profit']/1e9:.2f} bi" if latest_annual['gross_profit'] else "   Lucro Bruto: N/A")
        print(f"   EBIT: R$ {latest_annual['ebit']/1e9:.2f} bi" if latest_annual['ebit'] else "   EBIT: N/A")
        print(f"   Lucro Líquido: R$ {latest_annual['net_income']/1e9:.2f} bi" if latest_annual['net_income'] else "   Lucro Líquido: N/A")
else:
    print("⚠️ Nenhum dado de DRE disponível")

📡 Coletando DRE de PETR4...
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/external/brapi/petr4_income_statement.json
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/processed/petr4_income_statement.csv

📊 DRE:
   Total de registros: 79
   Anuais: 15 | Trimestrais: 64

   Última DRE Anual (2024-12-31):
   Receita: R$ 490.83 bi
   Lucro Bruto: R$ 246.46 bi
   EBIT: R$ 137.20 bi
   Lucro Líquido: R$ 37.01 bi


## 8. Coleta de Fluxo de Caixa — PETR4

**Endpoint**: `GET /api/quote/PETR4?modules=cashflowHistory,cashflowHistoryQuarterly`  
**Artefatos**:
- Raw: `data/external/brapi/petr4_cashflow.json`
- Processado: `data/processed/petr4_cashflow.csv`

### Campos extraídos:
- `operatingCashFlow`: FCO (Fluxo de Caixa Operacional)
- `investmentCashFlow`: FCI (Fluxo de Caixa de Investimentos)
- `financingCashFlow`: FCF (Fluxo de Caixa de Financiamentos)
- `increaseOrDecreaseInCash`: Variação Líquida de Caixa

In [8]:
# ==============================================================================
# CODEBLOCK 8: COLETA DE FLUXO DE CAIXA — PETR4
# ==============================================================================
# Artefatos:
#   - data/external/brapi/petr4_cashflow.json (raw)
#   - data/processed/petr4_cashflow.csv (processado)
# ==============================================================================

print(f"📡 Coletando DFC de {TICKER_PRINCIPAL}...")

# Módulos a serem coletados
modules = "cashflowHistory,cashflowHistoryQuarterly"

# Chamar endpoint
data = brapi_get(f"/quote/{TICKER_PRINCIPAL}", params={"modules": modules})

# Validar resposta
if "results" not in data or len(data["results"]) == 0:
    raise ValueError(f"Nenhum resultado retornado para {TICKER_PRINCIPAL}")

result = data["results"][0]

# Salvar JSON raw
save_json(data, PATH_EXTERNAL / "petr4_cashflow.json")

# Campos a extrair
cf_fields = [
    "endDate", "type", "operatingCashFlow", "investmentCashFlow",
    "financingCashFlow", "increaseOrDecreaseInCash"
]

# Processar histórico anual e trimestral
annual_data = result.get("cashflowHistory", [])
quarterly_data = result.get("cashflowHistoryQuarterly", [])

all_records = []

for record in annual_data:
    row = {field: record.get(field) for field in cf_fields}
    row["period_type"] = "annual"
    all_records.append(row)

for record in quarterly_data:
    row = {field: record.get(field) for field in cf_fields}
    row["period_type"] = "quarterly"
    all_records.append(row)

if all_records:
    df_cf = pd.DataFrame(all_records)
    
    # Renomear colunas
    df_cf = df_cf.rename(columns={
        "endDate": "end_date",
        "type": "report_type",
        "operatingCashFlow": "fco",
        "investmentCashFlow": "fci",
        "financingCashFlow": "fcf",
        "increaseOrDecreaseInCash": "cash_change"
    })
    
    # Ordenar por data
    df_cf = df_cf.sort_values("end_date", ascending=False).reset_index(drop=True)
    
    # Salvar CSV
    csv_path = PATH_PROCESSED / "petr4_cashflow.csv"
    df_cf.to_csv(csv_path, index=False)
    print(f"✅ Salvo: {csv_path}")
    
    # Exibir resumo
    print(f"\n📊 DFC:")
    print(f"   Total de registros: {len(df_cf)}")
    print(f"   Anuais: {len(annual_data)} | Trimestrais: {len(quarterly_data)}")
    
    # Última DFC anual
    latest_annual = df_cf[df_cf["period_type"] == "annual"].iloc[0] if len(annual_data) > 0 else None
    if latest_annual is not None:
        print(f"\n   Última DFC Anual ({latest_annual['end_date']}):")
        print(f"   FCO: R$ {latest_annual['fco']/1e9:.2f} bi" if latest_annual['fco'] else "   FCO: N/A")
        print(f"   FCI: R$ {latest_annual['fci']/1e9:.2f} bi" if latest_annual['fci'] else "   FCI: N/A")
        print(f"   FCF: R$ {latest_annual['fcf']/1e9:.2f} bi" if latest_annual['fcf'] else "   FCF: N/A")
else:
    print("⚠️ Nenhum dado de DFC disponível")

📡 Coletando DFC de PETR4...
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/external/brapi/petr4_cashflow.json
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/processed/petr4_cashflow.csv

📊 DFC:
   Total de registros: 79
   Anuais: 15 | Trimestrais: 64

   Última DFC Anual (2024-12-31):
   FCO: R$ 204.04 bi
   FCI: R$ -72.36 bi
   FCF: R$ -179.97 bi


## 9. Coleta de Histórico do IBOV (^BVSP) — Yahoo Finance

**Fonte:** Yahoo Finance via `yfinance`

O IBOV (^BVSP) requer autenticação na Brapi API. Utilizamos Yahoo Finance como fonte alternativa confiável para dados do índice IBOVESPA.

**Parâmetros:**
- `start`: data inicial (calculada a partir de DATA_RANGE)
- `end`: data atual
- `interval`: granularidade (1d, 1wk, 1mo)

**Artefatos Gerados:**
- `data/external/yfinance/ibov_historical.json` (raw)
- `data/processed/ibov_prices.csv` (processado)

In [9]:
# ==============================================================================
# CODEBLOCK 9: COLETA DE HISTÓRICO DO IBOV (^BVSP) — YAHOO FINANCE
# ==============================================================================
# Artefatos:
#   - data/external/yfinance/ibov_historical.json (raw)
#   - data/processed/ibov_prices.csv (processado)
# ==============================================================================

print(f"📡 Coletando histórico do IBOV ({TICKER_BENCHMARK}) via Yahoo Finance...")
print(f"   Período: {DATA_INICIO} a {DATA_REFERENCIA}")

# Baixar dados via yfinance
ibov = yf.Ticker(TICKER_BENCHMARK)
df_ibov = ibov.history(start=DATA_INICIO, end=DATA_REFERENCIA, interval=DATA_INTERVAL)

# Validar dados
if df_ibov.empty:
    raise ValueError(f"Nenhum dado retornado para {TICKER_BENCHMARK}")

# Reset index para ter date como coluna
df_ibov = df_ibov.reset_index()

# Converter para formato serializable JSON
ibov_data = {
    "ticker": TICKER_BENCHMARK,
    "source": "yahoo_finance",
    "period": {"start": DATA_INICIO, "end": DATA_REFERENCIA},
    "interval": DATA_INTERVAL,
    "total_records": len(df_ibov),
    "historicalDataPrice": df_ibov.to_dict(orient="records")
}

# Salvar JSON raw (com conversão de timestamps)
json_data = {
    "_metadata": {
        "collected_at": datetime.now().isoformat(),
        "data_referencia": DATA_REFERENCIA,
        "source": "yahoo_finance"
    },
    "ticker": TICKER_BENCHMARK,
    "total_records": len(df_ibov),
    "period": {"start": DATA_INICIO, "end": DATA_REFERENCIA}
}

json_path = PATH_EXTERNAL_YFINANCE / "ibov_historical.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2, default=str)
print(f"✅ Salvo: {json_path}")

# Processar para CSV
df_ibov_processed = df_ibov.copy()

# Renomear colunas
df_ibov_processed.columns = [col.lower().replace(" ", "_") for col in df_ibov_processed.columns]

# Converter date para string
if "date" in df_ibov_processed.columns:
    df_ibov_processed["date"] = pd.to_datetime(df_ibov_processed["date"]).dt.strftime("%Y-%m-%d")

# Selecionar colunas relevantes
cols_to_keep = ["date", "open", "high", "low", "close", "volume"]
df_ibov_processed = df_ibov_processed[[c for c in cols_to_keep if c in df_ibov_processed.columns]]

# Renomear para padrão ibov_
df_ibov_processed = df_ibov_processed.rename(columns={
    "open": "ibov_open",
    "high": "ibov_high",
    "low": "ibov_low",
    "close": "ibov_close",
    "volume": "ibov_volume"
})

# Ordenar por data
df_ibov_processed = df_ibov_processed.sort_values("date").reset_index(drop=True)

# Salvar CSV
csv_path = PATH_PROCESSED / "ibov_prices.csv"
df_ibov_processed.to_csv(csv_path, index=False)
print(f"✅ Salvo: {csv_path}")

# Exibir resumo
print(f"\n📊 Histórico IBOV (Yahoo Finance):")
print(f"   Total de registros: {len(df_ibov_processed)}")
print(f"   Período: {df_ibov_processed['date'].min()} a {df_ibov_processed['date'].max()}")
print(f"   Último fechamento: {df_ibov_processed['ibov_close'].iloc[-1]:,.0f} pts")
print(f"   Variação no período: {((df_ibov_processed['ibov_close'].iloc[-1] / df_ibov_processed['ibov_close'].iloc[0]) - 1) * 100:.2f}%")

📡 Coletando histórico do IBOV (^BVSP) via Yahoo Finance...
   Período: 2016-01-01 a 2025-12-06
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/external/yfinance/ibov_historical.json
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/processed/ibov_prices.csv

📊 Histórico IBOV (Yahoo Finance):
   Total de registros: 2468
   Período: 2016-01-04 a 2025-12-05
   Último fechamento: 157,369 pts
   Variação no período: 273.43%


## 9.1 Coleta de Taxa SELIC — BCB (SGS)

**Fonte:** Banco Central do Brasil - Sistema Gerenciador de Séries (SGS)

**Série:** 432 - Taxa SELIC Meta (% a.a.)

**API:** `https://api.bcb.gov.br/dados/serie/bcdata.sgs.{serie}/dados?formato=json`

**Artefatos Gerados:**
- `data/external/bcb/selic_meta.json` (raw)
- `data/processed/selic.csv` (processado)

In [10]:
# ==============================================================================
# CODEBLOCK 9.1: COLETA DE TAXA SELIC — BCB (SGS)
# ==============================================================================
# Artefatos:
#   - data/external/bcb/selic_meta.json (raw)
#   - data/processed/selic.csv (processado)
# ==============================================================================

print("📡 Coletando taxa SELIC do Banco Central (SGS)...")

# Série 432 = Taxa SELIC Meta (% a.a.)
SERIE_SELIC_META = 432

# Converter datas para formato BCB (dd/mm/yyyy)
data_inicio_bcb = datetime.strptime(DATA_INICIO, "%Y-%m-%d").strftime("%d/%m/%Y")
data_fim_bcb = datetime.strptime(DATA_REFERENCIA, "%Y-%m-%d").strftime("%d/%m/%Y")

# Obter SELIC Meta (anual) com retry
url_meta = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{SERIE_SELIC_META}/dados?formato=json&dataInicial={data_inicio_bcb}&dataFinal={data_fim_bcb}"
print(f"   URL: {url_meta}")

selic_meta_data = None
for attempt in range(3):
    try:
        response_meta = requests.get(url_meta, timeout=60)
        response_meta.raise_for_status()
        if response_meta.text.strip():
            selic_meta_data = response_meta.json()
            break
        else:
            print(f"   ⚠️ Tentativa {attempt+1}: resposta vazia, tentando novamente...")
    except Exception as e:
        print(f"   ⚠️ Tentativa {attempt+1}: {e}")
        if attempt < 2:
            import time
            time.sleep(2)

if not selic_meta_data:
    raise ValueError("Falha ao obter dados SELIC do BCB após 3 tentativas")

# Salvar JSON raw
json_data = {
    "_metadata": {
        "collected_at": datetime.now().isoformat(),
        "data_referencia": DATA_REFERENCIA,
        "source": "bcb.gov.br/SGS",
        "serie": SERIE_SELIC_META
    },
    "data": selic_meta_data
}

json_path = PATH_EXTERNAL_BCB / "selic_meta.json"
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(json_data, f, ensure_ascii=False, indent=2)
print(f"✅ Salvo: {json_path}")

# Processar para CSV
df_selic = pd.DataFrame(selic_meta_data)
df_selic["data"] = pd.to_datetime(df_selic["data"], format="%d/%m/%Y")
df_selic["valor"] = df_selic["valor"].astype(float)
df_selic = df_selic.rename(columns={"data": "date", "valor": "selic_meta_aa"})
df_selic = df_selic.sort_values("date").reset_index(drop=True)

# Calcular SELIC diária (aproximação)
# Formula: (1 + selic_aa/100)^(1/252) - 1
df_selic["selic_diaria"] = (1 + df_selic["selic_meta_aa"]/100) ** (1/252) - 1

# Salvar CSV
csv_path = PATH_PROCESSED / "selic.csv"
df_selic.to_csv(csv_path, index=False)
print(f"✅ Salvo: {csv_path}")

# Exibir resumo
print(f"\n📊 Taxa SELIC (BCB):")
print(f"   Total de registros: {len(df_selic)}")
print(f"   Período: {df_selic['date'].min().date()} a {df_selic['date'].max().date()}")
print(f"   SELIC atual: {df_selic['selic_meta_aa'].iloc[-1]:.2f}% a.a.")
print(f"   SELIC diária equivalente: {df_selic['selic_diaria'].iloc[-1]*100:.6f}%")
print(f"\n   Histórico recente:")
print(df_selic.tail().to_string(index=False))

📡 Coletando taxa SELIC do Banco Central (SGS)...
   URL: https://api.bcb.gov.br/dados/serie/bcdata.sgs.432/dados?formato=json&dataInicial=01/01/2016&dataFinal=06/12/2025
   ⚠️ Tentativa 1: Expecting value: line 1 column 1 (char 0)
   ⚠️ Tentativa 2: Expecting value: line 1 column 1 (char 0)
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/external/bcb/selic_meta.json
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/processed/selic.csv

📊 Taxa SELIC (BCB):
   Total de registros: 3628
   Período: 2016-01-01 a 2025-12-06
   SELIC atual: 15.00% a.a.
   SELIC diária equivalente: 0.055476%

   Histórico recente:
      date  selic_meta_aa  selic_diaria
2025-12-02           15.0      0.000555
2025-12-03           15.0      0.000555
2025-12-04           15.0      0.000555
2025-12-05           15.0      0.000555
2025-12-06           15.0      0.000555


## 10. Consolidação de Retornos

**Processamento:**
- Unificar preços de PETR4 e IBOV por data
- Calcular retornos logarítmicos diários
- Calcular retornos em excesso (Ri - Rf)
- Gerar série temporal unificada

**Artefatos Gerados:**
- `data/processed/returns.csv` (série de retornos consolidada)

In [11]:
# ==============================================================================
# CODEBLOCK 10: CONSOLIDAÇÃO DE RETORNOS COM RF REAL
# ==============================================================================
# Artefatos:
#   - data/processed/returns.csv (série de retornos consolidada)
# ==============================================================================

print("📊 Consolidando retornos com Rf real (SELIC)...")

# Carregar preços processados
df_petr4 = pd.read_csv(PATH_PROCESSED / "petr4_prices.csv", parse_dates=["date"])
df_ibov = pd.read_csv(PATH_PROCESSED / "ibov_prices.csv", parse_dates=["date"])
df_selic = pd.read_csv(PATH_PROCESSED / "selic.csv", parse_dates=["date"])

# Unificar PETR4 e IBOV por data
df_returns = df_petr4[["date", "close"]].rename(columns={"close": "petr4_price"})
df_returns = df_returns.merge(
    df_ibov[["date", "ibov_close"]].rename(columns={"ibov_close": "ibov_price"}),
    on="date",
    how="inner"
)

# Merge com SELIC (usando merge_asof para pegar a SELIC mais recente)
df_returns = df_returns.sort_values("date")
df_selic = df_selic.sort_values("date")
df_returns = pd.merge_asof(df_returns, df_selic[["date", "selic_diaria"]], on="date", direction="backward")

# Renomear para rf
df_returns = df_returns.rename(columns={"selic_diaria": "rf"})

# Ordenar por data
df_returns = df_returns.sort_values("date").reset_index(drop=True)

# Calcular retornos logarítmicos diários
df_returns["r_petr4"] = np.log(df_returns["petr4_price"] / df_returns["petr4_price"].shift(1))
df_returns["r_ibov"] = np.log(df_returns["ibov_price"] / df_returns["ibov_price"].shift(1))

# Calcular retornos em excesso (Ri - Rf)
df_returns["r_petr4_excess"] = df_returns["r_petr4"] - df_returns["rf"]
df_returns["r_ibov_excess"] = df_returns["r_ibov"] - df_returns["rf"]

# Remover primeira linha (NaN dos retornos)
df_returns = df_returns.dropna().reset_index(drop=True)

# Salvar CSV
csv_path = PATH_PROCESSED / "returns.csv"
df_returns.to_csv(csv_path, index=False)
print(f"✅ Salvo: {csv_path}")

# Resumo estatístico
print(f"\n📊 Série de Retornos:")
print(f"   Total de observações: {len(df_returns)}")
print(f"   Período: {df_returns['date'].min().date()} a {df_returns['date'].max().date()}")
print(f"\n   Retorno médio diário PETR4: {df_returns['r_petr4'].mean()*100:.4f}%")
print(f"   Retorno médio diário IBOV: {df_returns['r_ibov'].mean()*100:.4f}%")
print(f"   Volatilidade diária PETR4: {df_returns['r_petr4'].std()*100:.4f}%")
print(f"   Volatilidade diária IBOV: {df_returns['r_ibov'].std()*100:.4f}%")
print(f"   Taxa livre de risco média (Rf): {df_returns['rf'].mean()*100:.6f}%")
print(f"   Taxa livre de risco atual (Rf): {df_returns['rf'].iloc[-1]*100:.6f}%")

# Correlação
corr = df_returns["r_petr4"].corr(df_returns["r_ibov"])
print(f"\n   Correlação PETR4 x IBOV: {corr:.4f}")

# Retornos anualizados
dias = len(df_returns)
r_petr4_anual = (1 + df_returns["r_petr4"].mean()) ** 252 - 1
r_ibov_anual = (1 + df_returns["r_ibov"].mean()) ** 252 - 1
vol_petr4_anual = df_returns["r_petr4"].std() * np.sqrt(252)
vol_ibov_anual = df_returns["r_ibov"].std() * np.sqrt(252)

print(f"\n   📈 Métricas Anualizadas:")
print(f"   Retorno PETR4: {r_petr4_anual*100:.2f}% a.a.")
print(f"   Retorno IBOV: {r_ibov_anual*100:.2f}% a.a.")
print(f"   Volatilidade PETR4: {vol_petr4_anual*100:.2f}% a.a.")
print(f"   Volatilidade IBOV: {vol_ibov_anual*100:.2f}% a.a.")

📊 Consolidando retornos com Rf real (SELIC)...
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/processed/returns.csv

📊 Série de Retornos:
   Total de observações: 2467
   Período: 2016-01-05 a 2025-12-05

   Retorno médio diário PETR4: 0.0616%
   Retorno médio diário IBOV: 0.0534%
   Volatilidade diária PETR4: 2.8818%
   Volatilidade diária IBOV: 1.4825%
   Taxa livre de risco média (Rf): 0.035885%
   Taxa livre de risco atual (Rf): 0.055476%

   Correlação PETR4 x IBOV: 0.7187

   📈 Métricas Anualizadas:
   Retorno PETR4: 16.78% a.a.
   Retorno IBOV: 14.40% a.a.
   Volatilidade PETR4: 45.75% a.a.
   Volatilidade IBOV: 23.53% a.a.


## 11. Log de Validação e Resumo da Coleta

**Processamento:**
- Validar existência de todos os artefatos gerados
- Gerar log de coleta com timestamps
- Resumir status de cada endpoint

**Artefatos Gerados:**
- `data/outputs/collection_log.json` (log de validação)

In [12]:
# ==============================================================================
# CODEBLOCK 11: LOG DE VALIDAÇÃO E RESUMO DA COLETA
# ==============================================================================
# Artefatos:
#   - data/outputs/collection_log.json (log de validação)
# ==============================================================================

print("📋 Validando artefatos gerados...")

# Definir artefatos esperados
expected_artifacts = {
    "external_brapi": [
        "petr4_quote.json",
        "petr4_historical.json",
        "petr4_fundamental.json",
        "petr4_financial_data.json",
        "petr4_balance_sheet.json",
        "petr4_income_statement.json",
        "petr4_cashflow.json"
    ],
    "external_yfinance": [
        "ibov_historical.json"
    ],
    "external_bcb": [
        "selic_meta.json"
    ],
    "processed": [
        "petr4_prices.csv",
        "petr4_dividends.csv",
        "petr4_balance_sheet.csv",
        "petr4_income_statement.csv",
        "petr4_cashflow.csv",
        "ibov_prices.csv",
        "selic.csv",
        "returns.csv"
    ]
}

# Validar existência de cada artefato
validation_results = {
    "collection_timestamp": datetime.now().isoformat(),
    "config": {
        "ticker_principal": TICKER_PRINCIPAL,
        "ticker_benchmark": TICKER_BENCHMARK,
        "data_range": DATA_RANGE,
        "data_interval": DATA_INTERVAL,
        "data_inicio": DATA_INICIO,
        "brapi_base_url": BRAPI_BASE_URL,
        "sources": {
            "petr4": "brapi.dev",
            "ibov": "yahoo_finance",
            "selic": "bcb.gov.br/SGS"
        }
    },
    "artifacts": {
        "external_brapi": [],
        "external_yfinance": [],
        "external_bcb": [],
        "processed": []
    },
    "summary": {
        "total_expected": 0,
        "total_found": 0,
        "total_missing": 0,
        "status": "UNKNOWN"
    }
}

# Verificar arquivos em external/brapi
for filename in expected_artifacts["external_brapi"]:
    filepath = PATH_EXTERNAL / filename
    exists = filepath.exists()
    size = filepath.stat().st_size if exists else 0
    
    validation_results["artifacts"]["external_brapi"].append({
        "filename": filename,
        "path": str(filepath),
        "exists": exists,
        "size_bytes": size,
        "size_formatted": f"{size/1024:.2f} KB" if exists else "N/A"
    })

# Verificar arquivos em external/yfinance
for filename in expected_artifacts["external_yfinance"]:
    filepath = PATH_EXTERNAL_YFINANCE / filename
    exists = filepath.exists()
    size = filepath.stat().st_size if exists else 0
    
    validation_results["artifacts"]["external_yfinance"].append({
        "filename": filename,
        "path": str(filepath),
        "exists": exists,
        "size_bytes": size,
        "size_formatted": f"{size/1024:.2f} KB" if exists else "N/A"
    })

# Verificar arquivos em external/bcb
for filename in expected_artifacts["external_bcb"]:
    filepath = PATH_EXTERNAL_BCB / filename
    exists = filepath.exists()
    size = filepath.stat().st_size if exists else 0
    
    validation_results["artifacts"]["external_bcb"].append({
        "filename": filename,
        "path": str(filepath),
        "exists": exists,
        "size_bytes": size,
        "size_formatted": f"{size/1024:.2f} KB" if exists else "N/A"
    })

# Verificar arquivos em processed
for filename in expected_artifacts["processed"]:
    filepath = PATH_PROCESSED / filename
    exists = filepath.exists()
    size = filepath.stat().st_size if exists else 0
    
    validation_results["artifacts"]["processed"].append({
        "filename": filename,
        "path": str(filepath),
        "exists": exists,
        "size_bytes": size,
        "size_formatted": f"{size/1024:.2f} KB" if exists else "N/A"
    })

# Calcular estatísticas
total_expected = (len(expected_artifacts["external_brapi"]) + 
                  len(expected_artifacts["external_yfinance"]) +
                  len(expected_artifacts["external_bcb"]) +
                  len(expected_artifacts["processed"]))
total_found = (sum(1 for a in validation_results["artifacts"]["external_brapi"] if a["exists"]) +
               sum(1 for a in validation_results["artifacts"]["external_yfinance"] if a["exists"]) +
               sum(1 for a in validation_results["artifacts"]["external_bcb"] if a["exists"]) +
               sum(1 for a in validation_results["artifacts"]["processed"] if a["exists"]))
total_missing = total_expected - total_found

validation_results["summary"]["total_expected"] = total_expected
validation_results["summary"]["total_found"] = total_found
validation_results["summary"]["total_missing"] = total_missing
validation_results["summary"]["status"] = "SUCCESS" if total_missing == 0 else "INCOMPLETE"

# Salvar log
log_path = PATH_OUTPUTS / "collection_log.json"
with open(log_path, "w", encoding="utf-8") as f:
    json.dump(validation_results, f, ensure_ascii=False, indent=2)
print(f"✅ Salvo: {log_path}")

# Exibir resumo
print("\n" + "=" * 60)
print("📊 RESUMO DA COLETA DE DADOS")
print("=" * 60)

print(f"\n🔧 Configuração:")
print(f"   Ticker Principal: {TICKER_PRINCIPAL} (Brapi)")
print(f"   Benchmark: {TICKER_BENCHMARK} (Yahoo Finance)")
print(f"   Taxa Livre de Risco: SELIC (BCB/SGS)")
print(f"   Período: {DATA_RANGE} ({DATA_INICIO} a {DATA_REFERENCIA})")
print(f"   Intervalo: {DATA_INTERVAL}")

print(f"\n📁 Artefatos em data/external/brapi/:")
for a in validation_results["artifacts"]["external_brapi"]:
    status = "✅" if a["exists"] else "❌"
    print(f"   {status} {a['filename']} ({a['size_formatted']})")

print(f"\n📁 Artefatos em data/external/yfinance/:")
for a in validation_results["artifacts"]["external_yfinance"]:
    status = "✅" if a["exists"] else "❌"
    print(f"   {status} {a['filename']} ({a['size_formatted']})")

print(f"\n📁 Artefatos em data/external/bcb/:")
for a in validation_results["artifacts"]["external_bcb"]:
    status = "✅" if a["exists"] else "❌"
    print(f"   {status} {a['filename']} ({a['size_formatted']})")

print(f"\n📁 Artefatos em data/processed/:")
for a in validation_results["artifacts"]["processed"]:
    status = "✅" if a["exists"] else "❌"
    print(f"   {status} {a['filename']} ({a['size_formatted']})")

print(f"\n📈 Estatísticas:")
print(f"   Total esperado: {total_expected}")
print(f"   Total encontrado: {total_found}")
print(f"   Total faltando: {total_missing}")

status_emoji = "✅" if validation_results["summary"]["status"] == "SUCCESS" else "⚠️"
print(f"\n{status_emoji} Status Final: {validation_results['summary']['status']}")
print("=" * 60)

# Análise de completude para ROADMAP
print("\n" + "=" * 60)
print("📋 ANÁLISE DE DADOS PARA ROADMAP Q-VAL")
print("=" * 60)

print("\n✅ Dados suficientes para:")
print("   • Fase 2: Cálculo de Retornos e Estatísticas Descritivas")
print("   • Fase 3: Estimação CAPM (Beta, Ke)")
print("   • Fase 5: Motor Q-VAL (métricas de Valor, Qualidade, Risco)")
print("   • Fase 6: Análise de Valuation e Mispricing (ICC vs CAPM)")

print("\n📊 Fontes de Dados:")
print("   • PETR4 (preços, dividendos, demonstrações): Brapi API")
print("   • IBOV (benchmark): Yahoo Finance")
print("   • SELIC (taxa livre de risco): BCB/SGS")

print("\n⚠️ Observações:")
print("   • Beta calculável via regressão PETR4 vs IBOV")
print("   • ROE, ROA, margens disponíveis em petr4_financial_data.json")
print("   • Balanço, DRE e DFC disponíveis para múltiplos períodos")
print("=" * 60)

📋 Validando artefatos gerados...
✅ Salvo: /home/subtilizer/Workspace/unb-cca-mqac/data/outputs/collection_log.json

📊 RESUMO DA COLETA DE DADOS

🔧 Configuração:
   Ticker Principal: PETR4 (Brapi)
   Benchmark: ^BVSP (Yahoo Finance)
   Taxa Livre de Risco: SELIC (BCB/SGS)
   Período: 10y (2016-01-01 a 2025-12-06)
   Intervalo: 1d

📁 Artefatos em data/external/brapi/:
   ✅ petr4_quote.json (1.06 KB)
   ✅ petr4_historical.json (522.61 KB)
   ✅ petr4_fundamental.json (25.78 KB)
   ✅ petr4_financial_data.json (5.08 KB)
   ✅ petr4_balance_sheet.json (201.43 KB)
   ✅ petr4_income_statement.json (170.41 KB)
   ✅ petr4_cashflow.json (47.97 KB)

📁 Artefatos em data/external/yfinance/:
   ✅ ibov_historical.json (0.25 KB)

📁 Artefatos em data/external/bcb/:
   ✅ selic_meta.json (225.31 KB)

📁 Artefatos em data/processed/:
   ✅ petr4_prices.csv (122.67 KB)
   ✅ petr4_dividends.csv (1.98 KB)
   ✅ petr4_balance_sheet.csv (11.02 KB)
   ✅ petr4_income_statement.csv (9.04 KB)
   ✅ petr4_cashflow.csv (6.